In [1]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup

# -----------------------------
# 1️⃣ Criar pasta se não existir
# -----------------------------
output_folder = "/home/nuno/Desktop/_TEMP/vela"
os.makedirs(output_folder, exist_ok=True)

# -----------------------------
# 2️⃣ Obter HTML da página
# -----------------------------
url = "https://www.sailwave.com/results/POR-Clubes/1PAN_ILCA_Oeiras.htm"
html = requests.get(url).content
soup = BeautifulSoup(html, "html.parser")
site_url = url

# -----------------------------
# 3️⃣ Encontrar todas as tabelas summarytable e títulos
# -----------------------------
tables = soup.find_all("table", class_="summarytable")
titles = soup.find_all("h3", class_="summarytitle")

competicoes = {}

# -----------------------------
# 4️⃣ Processar cada competição
# -----------------------------
for i, table in enumerate(tables):
    nome_competicao = titles[i].get_text(strip=True) if i < len(titles) else f"Competicao_{i+1}"

    headers = [th.get_text(strip=True) for th in table.find_all("th")]
    rows = []
    for tr in table.find_all("tr"):
        cols = [td.get_text(strip=True) for td in tr.find_all("td")]
        if cols:
            rows.append(cols)
    df = pd.DataFrame(rows, columns=headers)

    # Corrigir Rank geral
    if df.columns[0] == "" or df.columns[0].lower() == "rank":
        df = df.rename(columns={df.columns[0]: "classif_geral"})

    # -----------------------------
    # 4a️⃣ Manter R1-R4 como strings para preservar descartes
    colunas_r = ["R1","R2","R3","R4"]
    for c in colunas_r:
        if c in df.columns:
            df[c] = df[c].astype(str)

    # Converter apenas Total e Nett para numérico
    for c in ["Total","Nett"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # Ranking interno por escalão
    df["classif_no_escalao"] = df.sort_values("Nett").groupby("Division").cumcount() + 1
    # Ranking interno por clube
    df["Rank_Clube"] = df.sort_values("Nett").groupby("Yacht Club").cumcount() + 1

    # Classificação por division
    por_divisao = {div: dados.sort_values("Nett").reset_index(drop=True)
                   for div, dados in df.groupby("Division")}
    # Classificação por clube
    por_clube = {clube: dados.sort_values("Nett").reset_index(drop=True)
                 for clube, dados in df.groupby("Yacht Club")}

    # Classificação coletiva por clubes (Top 3)
    N = 3
    class_coletiva = (
        df[df["Rank_Clube"] <= N]
        .groupby("Yacht Club")
        .agg(Pontos_Coletivos=("Nett", "sum"),
             Atletas_Contam=("HelmName", "count"))
        .sort_values("Pontos_Coletivos")
        .reset_index()
    )
    class_coletiva["Rank_Coletivo"] = range(1, len(class_coletiva)+1)

    competicoes[nome_competicao] = {
        "df": df,
        "por_divisao": por_divisao,
        "por_clube": por_clube,
        "coletiva_clubes": class_coletiva
    }

# -----------------------------
# 5️⃣ Gerar página HTML apontando para estilos.css
# -----------------------------
html_content = '<html><head><title>Classificações Vela</title>'
html_content += '<link rel="stylesheet" type="text/css" href="estilos_south.css">'
html_content += '</head><body>'

# Índice
html_content += '<h1 id="topo">Índice</h1>'
for comp in competicoes.keys():
    html_content += f'<li><a href="#{comp}">{comp}</a></li>'
html_content += '<li><a href="#ranking_clubes_total">Ranking clubes total</a></li>'
html_content += "</ul>"

# -----------------------------
# Função para aplicar cores alternadas via BeautifulSoup
# -----------------------------
def html_table_with_stripes(df):
    html_table = df.to_html(index=False, escape=False, classes="summarytable")
    soup_table = BeautifulSoup(html_table, "html.parser")
    # Aplica even/odd às linhas do tbody
    for i, tr in enumerate(soup_table.find_all("tr")[1:]):  # ignora header
        tr['class'] = 'even' if i % 2 == 0 else 'odd'
    return str(soup_table)

# -----------------------------
# Inserir competições
# -----------------------------
for comp, data in competicoes.items():
    
    # Escolher cor de fundo diferente por competição
    if "ILCA4" in comp.upper():
        html_content += f'<div id="ilca4">'
    elif "ILCA6" in comp.upper():
        html_content += f'<div id="ilca6">'
    else:
        html_content += f'<div>'
    html_content += f'<hr><h2 id="{comp}">{comp}</h2>'

    # Tabela original (remover colunas extras)
    df_orig = data["df"].drop(columns=["classif_no_escalao", "Rank_Clube"])
    #html_content += "<h3>Tabela original</h3>"
    html_content += f'<a href="{site_url}" target="_blank"><h3>Tabela original / Oficial</h3></a>'
    html_content += '<p><a href="#topo">Voltar ao topo</a></p>'
    html_content += html_table_with_stripes(df_orig)
    
    
    

    # Ranking por division (destacar top 3)
    
 
    
    html_content += f'<h3>Ranking por Division</h3>'
    for div, df_div in data["por_divisao"].items():
        df_div_clean = df_div.drop(columns=["Rank_Clube"])
        df_div_clean['classif_no_escalao'] = df_div_clean['classif_no_escalao'].astype(object)
        for i, row in df_div_clean.iterrows():
            if row['classif_no_escalao'] == 1: 
                df_div_clean.at[i, 'classif_no_escalao'] = f'<span class="place1">{row["classif_no_escalao"]}</span>'
            elif row['classif_no_escalao'] == 2: 
                df_div_clean.at[i, 'classif_no_escalao'] = f'<span class="place2">{row["classif_no_escalao"]}</span>'
            elif row['classif_no_escalao'] == 3: 
                df_div_clean.at[i, 'classif_no_escalao'] = f'<span class="place3">{row["classif_no_escalao"]}</span>'
        html_content += f'<h4>{div}</h4>'
        html_content += html_table_with_stripes(df_div_clean)

    # Ranking por clube
    html_content += "<h3>Ranking por Clube</h3>"
    for clube, df_cl in data["por_clube"].items():
        df_cl_clean = df_cl.drop(columns=["classif_no_escalao", "Rank_Clube"])
        html_content += f'<h4>{clube}</h4>'
        html_content += html_table_with_stripes(df_cl_clean)
        
    html_content += '</div>'  # fecha a div da competição

# Ranking coletivo final dos clubes
coletiva_total = pd.DataFrame()
for data in competicoes.values():
    df_co = data["coletiva_clubes"][["Yacht Club","Pontos_Coletivos"]].copy()
    coletiva_total = pd.concat([coletiva_total, df_co])
coletiva_total = coletiva_total.groupby("Yacht Club").sum().reset_index()
coletiva_total = coletiva_total.sort_values("Pontos_Coletivos").reset_index(drop=True)
coletiva_total["Rank_Coletivo_Total"] = range(1, len(coletiva_total)+1)

html_content += '<h2 id="ranking_clubes_total">Ranking clubes total</h2>'
html_content += '<p><a href="#topo">Voltar ao topo</a></p>'
html_content += html_table_with_stripes(coletiva_total)

# -----------------------------
# Salvar HTML
# -----------------------------
output_file = os.path.join(output_folder, "classificacoes_vela.html")
with open(output_file, "w", encoding="utf-8") as f:
    f.write(html_content)

print(f"Página HTML criada em: {output_file} (usa estilos.css para o CSS)")


Página HTML criada em: /home/nuno/Desktop/_TEMP/vela/classificacoes_vela.html (usa estilos.css para o CSS)
